In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

In [4]:
target_url = "https://books.toscrape.com/catalogue/page-{}.html"

In [5]:
books_data = []

In [7]:
# Loop every page including new ones

page = 1
while True:
    url = target_url.format(page)
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}. Fetching {page} failed.")
        break

    # If page doesn't exist, break
    if response.status_code == 404:
        break

    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")

    for book in books:
        try:

            title = book.h3.a["title"] if book.h3 and book.h3.a else "N/A"


            rating_class = book.p["class"][1] if book.p and len(book.p["class"]) > 1 else "Zero"
            rating_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5, "Zero": 0}
            rating = rating_dict.get(rating_class, 0)


            price_tag = book.find("p", class_="price_color")


            price_tag = book.find("p", class_="price_color")
            price = price_tag.text.strip() if price_tag else "N/A"
            price = re.sub(r'[^\d.,£]', '', price)  # Remove undesired char


            availability_tag = book.find("p", class_="instock availability")
            availability = availability_tag.text.strip() if availability_tag else "N/A"


            books_data.append([title, rating, price, availability])
        except Exception as e:
            print(f"Error raised by the book: {e}")

    # Wait 2 before going to next page
    time.sleep(2)
    page += 1

# Save in csv format
csv_filename = "books.csv"
try:
    with open(csv_filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Title", "Star", "Price", "Availability"])
        writer.writerows(books_data)
    print(f"Scraping completed. Data salved in {csv_filename}")
except IOError as e:
    print(f"Error in saving CSV: {e}")

Network error: 404 Client Error: Not Found for url: https://books.toscrape.com/catalogue/page-51.html. Fetching 51 failed.
Scraping completed. Data salved in books.csv
